In [3]:
import numpy as np

from FeatureAcquisition import FeatureAcquisition
from Predictor import Predictor
from Generators import GaussianSampler

In [4]:
x = np.array([[2., 4., 3., 5., 6., 3., 3., 4.]])
m = np.array([[0., 0., 1., 1., 0., 0., 0., 0.]])
cost_per_feature = np.ones(8)
budget = 4

feature_dim = 8
num_classes = 3
hidden_dim = 8
predictor = Predictor(feature_dim, num_classes, hidden_dim)

generator = GaussianSampler(mean=0.0, std=1.0)

FA = FeatureAcquisition(
    x=x,
    m=m,
    generative_model=generator,
    num_samples=10,
    predictor=predictor,
    alpha=1.0,
    gamma=0.0
)

# 초기 mask 확인
print(FA.m)

# mask 하나씩 잘 고르는지 확인
for t in range(5):
    scores = FA.alpha_gamma_cmi()
    print(FA.x, FA.m, FA.acquire())

[[0. 0. 1. 1. 0. 0. 0. 0.]]
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[0. 0. 1. 1. 0. 1. 0. 0.]] (array([[0., 0., 1., 1., 0., 1., 0., 0.]]), array([5]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[0. 0. 1. 1. 0. 1. 1. 0.]] (array([[0., 0., 1., 1., 0., 1., 1., 0.]]), array([6]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 0. 1. 1. 0. 1. 1. 0.]] (array([[1., 0., 1., 1., 0., 1., 1., 0.]]), array([0]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 1. 1. 1. 0. 1. 1. 0.]] (array([[1., 1., 1., 1., 0., 1., 1., 0.]]), array([1]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 1. 1. 1. 1. 1. 1. 0.]] (array([[1., 1., 1., 1., 1., 1., 1., 0.]]), array([4]))
